# Test 11 (paper) - Experimento 7 (PT)

Entrenamiento de Mask R-CNN con dataset modificado para emular "Test 11"

### Hiperparametros
* **epoch = 100**
    * steps x epoch = 146 (lotes de imagenes)
    * batch = 2
* optimizador = SGD
* Funcion de perdida = SMOOTHL1LOSS
* Metrica de evaluacion = mAP (IoU >= 0.5)
* **Mini-mask shape: 28x28**
* **RPN anchor scales: (8, 16, 32, 64, 128)**
* Tasa de aprendizaje: 0.001
* **imagenes = 305**
    * entrenamiento 70% = 177
    * validacion 30% = 128
* etiquetas = 9140
* **resolucion = 1920 x 1080**
* etiquetas = bounding box formato VOC XML
* **numero de clases = 2 (arandano, arandano-maduro)**
* **data augmentation = false**


## Comprobar directorio principal

In [1]:
!pwd && ls -l

/tf/PT_JoseVeloso/Mask_RCNN-master_matterport/model-training
total 1536
drwxr-xr-x 4 root root   4096 Aug  7 06:10 build
drwxr-xr-x 2 root root   4096 Aug  7 06:10 dist
drwxr-xr-x 2 root root   4096 Aug  8 03:15 mask_rcnn.egg-info
-rw-r--r-- 1 root root  25109 Sep 16 03:35 master-test_11_fase_2.ipynb
-rw-r--r-- 1 root root 350337 Sep 15 00:38 master-test_5-fase_2.ipynb
-rw-r--r-- 1 root root 162975 Sep 15 02:26 master-test_5-fine_tuning_1.ipynb
-rw-r--r-- 1 root root 162983 Sep 15 02:23 master-test_5.ipynb
-rw-r--r-- 1 root root 350111 Sep 15 02:26 master-test_5_fase_2.ipynb
-rw-r--r-- 1 root root 110478 Sep 15 02:15 master-test_6-fine_tuning_1.ipynb
-rw-r--r-- 1 root root 110254 Sep 15 02:25 master-test_6.ipynb
-rw-r--r-- 1 root root 270148 Sep 15 15:04 master-test_6_fase_2.ipynb
drwxr-xr-x 4 root root   4096 Sep 13 07:49 mrcnn
drwxr-xr-x 3 root root   4096 Sep 15 02:27 old


# Importar bibliotecas

In [2]:
# bibliotecas basicas
import os
from os import listdir
import sys
import json
import datetime

#sys.path.append("/tf/PT_JoseVeloso/Mask_RCNN-master/")

# bibliotecas avanzadas 
from xml.etree import ElementTree
import skimage.draw
import cv2
import imgaug

# bibliotecas mask rcnn 
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
from mrcnn import visualize

# biblioteca matplotlib 
import matplotlib.pyplot as plt

# bibliotecas numpy 
import numpy as np
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from numpy import mean

# bibliotecas keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img   #keras.preprocessing.image tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

# ignorar alertas de elementos que seran descontinuados
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

%matplotlib inline
#plt.show()

import imgaug.augmenters as iaa

# Fase 2 - Entrenamiento con dos clases y etiquetas de Bounding Box

En este entremamiento se utiliza un conjunto de datos simple con imágenes etiquetadas con cuadros delimitadores y una clase llamada 'Daño'. En la siguiente sección se encuentra el código para el entrenamiento del modelo. Se incluyen comentarios para describir mejor el flujo del programa.

In [ ]:
class DatasetArandanos(Dataset):
    
    # load_dataset function is used to load the train and test dataset
    def load_dataset(self, dataset_dir, is_train=True):
        
        # we use add_class for each class in our dataset and assign numbers to them. 0 is background
        # self.add_class('source', 'class id', 'class name')
        self.add_class("dataset", 1, "arandano")
        self.add_class("dataset", 2, "arandano-maduro")
        #self.add_class("dataset", 3, "Level-3")
        #self.add_class("dataset", 4, "Level-4")
        
        # we concatenate the dataset_dir with /images and /annots
        images_dir = dataset_dir + '/images/'
        annotations_dir = dataset_dir + '/annots/'
        
        # is_train will be true if we our training our model and false when we are testing the model
        for filename in listdir(images_dir):
            
            # extract image id
            image_id = filename[:-4] # used to skip last 4 chars which is '.jpg' (class_id.jpg)
            
            # if is_train is True skip all images with id greater than and equal to 160
             # roughly 80% of dataset for training
            if is_train and int(image_id) >= 11074 :
                #print("image_id: ", image_id)
                continue
             
            # if is_train is not True skip all images with id less than 420
            if not is_train and int(image_id) < 11074:
                continue
            
            # img_path and ann_path variables are defined
            img_path = images_dir + filename
            ann_path = annotations_dir + image_id + '.xml'
            
            # using add_image function we pass image_id, image_path and ann_path so that the current
            # image is added to the dataset for training
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

    # function used to extract bouding boxes from annotated files
    def extract_boxes(self, filename):

        # you can see how the images are annotated we extracrt the width, height and bndbox values

        # <size>

        #       <width>640</width>

        #       <height>360</height>

        #       <depth>3</depth>

        # </size>


        # <object>

        #          <name>damage</name>

        #          <pose>Unspecified</pose>

        #          <truncated>0</truncated>

        #          <difficult>0</difficult>


        #          <bndbox>

        #                 <xmin>315</xmin>

        #                 <ymin>160</ymin>

        #                 <xmax>381</xmax>

        #                 <ymax>199</ymax>

        #          </bndbox>

        # </object>

        # </annotation>
        
        # used to parse the .xml files
        tree = ElementTree.parse(filename)
        
        # to get the root of the xml file
        root = tree.getroot()
        
        # we will append all x, y coordinated in boxes
        # for all instances of an onject
        boxes = list()
        
        # we find all attributes with name bndbox
        # bndbox will exist for each ground truth in an image
        for box in root.findall('.//object'):
            name = box.find('name').text
            xmin = int(box.find('./bndbox/xmin').text)
            ymin = int(box.find('./bndbox/ymin').text)
            xmax = int(box.find('./bndbox/xmax').text)
            ymax = int(box.find('./bndbox/ymax').text)
            coors = [xmin, ymin, xmax, ymax, name]
            boxes.append(coors)
            
            # I have included this line to skip any un-annotated images
            if name=='arandano' or name=='arandano-maduro':
                boxes.append(coors)

        # extract width and height of the image
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        
        # return boxes-> list, width-> int and height-> int 
        return boxes, width, height
    
    # this function calls on the extract_boxes method and is used to load a mask for each instance in an image
    # returns a boolean mask with following dimensions width * height * instances        
    def load_mask(self, image_id):
        
        # info points to the current image_id
        info = self.image_info[image_id]
        
        # we get the annotation path of image_id which is dataset_dir/annots/image_id.xml
        path = info['annotation']
        
        # we call the extract_boxes method(above) to get bndbox from .xml file
        boxes, w, h = self.extract_boxes(path)
        
        # we create len(boxes) number of masks of height 'h' and width 'w'
        masks = zeros([h, w, len(boxes)], dtype='uint8')

        class_ids = list()
        
        # we loop over all boxes and generate masks (bndbox mask) and class id for each instance
        # masks will have rectange shape as we have used bndboxes for annotations
        # for example: if 2.jpg have four objects we will have following masks and class_ids
        # 000000000 000000000 000003330 111100000
        # 000011100 022200000 000003330 111100000
        # 000011100 022200000 000003330 111100000
        # 000000000 022200000 000000000 000000000
        #    1         2          3         1<- class_ids
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            
            # box[4] will have the name of the class for a particular damage
            if (box[4] == 'arandano'):
                masks[row_s:row_e, col_s:col_e, i] = 1
                class_ids.append(self.class_names.index('arandano'))
            elif(box[4] == 'arandano-maduro'):
                masks[row_s:row_e, col_s:col_e, i] = 2
                class_ids.append(self.class_names.index('arandano-maduro')) 
                
        # return masks and class_ids as array
        return masks, asarray(class_ids, dtype='int32')
    
    # this functions takes the image_id and returns the path of the image
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']


In [3]:

# damage configuration class, you can change values of hyper parameters here
class ConfigArandanos(Config):

    # nombre de la configuracion
    NAME = "arandano_cfg_test_11_fase_2_"    
    
    # clase arandano + clase background + 4 clases
    NUM_CLASSES = 1 + 2
    
    # pasos por epoch y confianza minima    # STEPS_PER_EPOCH = cantidad de lotes/batchs
    STEPS_PER_EPOCH = 61  # por epoch se entrenaran 61 lotes de 5 imagenes, dataset = 305

    # tasa de aprendizaje y momentum
    LEARNING_RATE=0.001
    LEARNING_MOMENTUM = 0.8
    
    # penalización de regularización
    WEIGHT_DECAY = 0.0001
    
    # el tamaño de la imagen está controlado por este parámetro
    IMAGE_MIN_DIM = 512
    
    # pasos de validación
    VALIDATION_STEPS = 50
    
    # número de regiones de interés generadas por imagen
    Train_ROIs_Per_Image = 200
    
    # escala de anclas RPN y proporciones (ratios) para encontrar la ROI
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)    # Longitud del lado del ancla cuadrada, en píxeles 
    RPN_ANCHOR_RATIOS = [0.5, 1, 1.5]   # Proporciones de anclas por cada celda (ancho/alto). Un valor de 1 representa un ancla cuadrada y 0,5 es un ancla ancha 

    #DEVICE = "/cpu:0"  # /cpu:0 or /gpu:0    
    DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

    IMAGES_PER_GPU = 2
    
    MINI_MASK_SHAPE = (28, 28)
    
ConfigArandanos().display()



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
DEVICE                         /gpu:0
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.8
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE 

### Entrenamiento

In [4]:
cd /tf/PT_JoseVeloso/Mask_RCNN-master_matterport/

/tf/PT_JoseVeloso/Mask_RCNN-master_matterport


In [5]:
#pesos = 'mask_rcnn_coco.h5'
#pesos = 'mask_rcnn_damage_cfg_0049.h5'
#pesos = 'arandano_cfg_test_5_20220907T0914/mask_rcnn_arandano_cfg_0300.h5'
#pesos = 'arandano_cfg_test_5_20220907T0914/mask_rcnn_arandano_cfg_0300.h5'
pesos = 'arandano_cfg_test_6_fase_2_20220915T0216/mask_rcnn_arandano_cfg_test_6_fase_2__0100.h5'

conjunto_datos = 'customImages/test_11_fase_2'


In [ ]:
# cargar dataset de entrenamiento
train_set = DatasetArandanos()
train_set.load_dataset(conjunto_datos, is_train=True)
train_set.prepare()

# cargar dataset de test 
test_set = DatasetArandanos()
test_set.load_dataset(conjunto_datos, is_train=False)
test_set.prepare()

# preparar la configuración llamando a la clase de configuración definida por el usuario
config = ConfigArandanos()

# definir el modelo
with tf.device(config.DEVICE):
    model = MaskRCNN(mode='training', model_dir='./', config=config)

# cargar pesos del modelo 
weights_path = pesos

# cargar los pesos del modelo
model.load_weights(weights_path, 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

#augmentation = iaa.Sequential(iaa.Fliplr(0.5), iaa.Flipud(0.5))
#augmentation = iaa.Sometimes(5/6,iaa.Fliplr(0.5), iaa.Flipud(0.5))
#augmentation = iaa.Sequential([iaa.Fliplr(0.5), iaa.Flipud(0.5)])
#augmentation = iaa.Sometimes(5/6, iaa.Fliplr(0.5), iaa.Flipud(0.5))
#augmentation = iaa.Sometimes(0.5, iaa.Fliplr(0.5), iaa.Flipud(0.5))

# start the training of model
# you can change epochs and layers (head or all)
#model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=100, layers='all', augmentation=augmentation)
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=100, layers='all')



Starting at epoch 0. LR=0.001

Checkpoint Path: ./arandano_cfg_test_11_fase_2_20220916T0336/mask_rcnn_arandano_cfg_test_11_fase_2__{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_branch2b         

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/100


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/sub:0", shape=(None,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/GatherV2_2:0", shape=(None, 7, 7, 256), dtype=float32), dense_shape=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/Shape:0", shape=(4,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/sub_1:0", shape=(None,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/roi_align_classifier/concat_grad/GatherV2_5:0", shape=(Non

61/61 [==============================] - 383s 6s/step - batch: 30.0000 - size: 2.0000 - loss: 4.5276 - rpn_class_loss: 0.3208 - rpn_bbox_loss: 2.1583 - mrcnn_class_loss: 0.0973 - mrcnn_bbox_loss: 1.3937 - mrcnn_mask_loss: 0.5574 - val_loss: 2.5070 - val_rpn_class_loss: 0.2687 - val_rpn_bbox_loss: 0.8192 - val_mrcnn_class_loss: 0.1705 - val_mrcnn_bbox_loss: 0.7376 - val_mrcnn_mask_loss: 0.5110
Epoch 2/100
61/61 [==============================] - 293s 5s/step - batch: 30.0000 - size: 2.0000 - loss: 1.8865 - rpn_class_loss: 0.0953 - rpn_bbox_loss: 0.6682 - mrcnn_class_loss: 0.0834 - mrcnn_bbox_loss: 0.5546 - mrcnn_mask_loss: 0.4850 - val_loss: 2.1111 - val_rpn_class_loss: 0.1831 - val_rpn_bbox_loss: 0.7234 - val_mrcnn_class_loss: 0.1726 - val_mrcnn_bbox_loss: 0.5394 - val_mrcnn_mask_loss: 0.4925
Epoch 3/100
61/61 [==============================] - 267s 4s/step - batch: 30.0000 - size: 2.0000 - loss: 1.6548 - rpn_class_loss: 0.0845 - rpn_bbox_loss: 0.5630 - mrcnn_class_loss: 0.1614 - mrcnn

61/61 [==============================] - 361s 6s/step - batch: 30.0000 - size: 2.0000 - loss: 0.4863 - rpn_class_loss: 0.0153 - rpn_bbox_loss: 0.0292 - mrcnn_class_loss: 0.0668 - mrcnn_bbox_loss: 0.0700 - mrcnn_mask_loss: 0.3049 - val_loss: 2.6560 - val_rpn_class_loss: 0.3438 - val_rpn_bbox_loss: 0.8280 - val_mrcnn_class_loss: 0.5545 - val_mrcnn_bbox_loss: 0.4547 - val_mrcnn_mask_loss: 0.4749
Epoch 42/100
61/61 [==============================] - 276s 5s/step - batch: 30.0000 - size: 2.0000 - loss: 0.4701 - rpn_class_loss: 0.0158 - rpn_bbox_loss: 0.0270 - mrcnn_class_loss: 0.0577 - mrcnn_bbox_loss: 0.0681 - mrcnn_mask_loss: 0.3017 - val_loss: 2.4152 - val_rpn_class_loss: 0.2346 - val_rpn_bbox_loss: 0.7686 - val_mrcnn_class_loss: 0.5478 - val_mrcnn_bbox_loss: 0.4473 - val_mrcnn_mask_loss: 0.4168
Epoch 43/100
61/61 [==============================] - 284s 5s/step - batch: 30.0000 - size: 2.0000 - loss: 0.4568 - rpn_class_loss: 0.0132 - rpn_bbox_loss: 0.0267 - mrcnn_class_loss: 0.0576 - mrc